In [19]:
import pandas as pd 
import numpy as np 
import math
import matplotlib.pyplot as plt
import scipy.stats
from scipy.stats import skew, norm
from scipy.optimize import fsolve
plt.style.use('seaborn')

Here we read in the front month WTI futures price from 01/01/2018-09/04/2021

In [20]:
df = pd.read_excel("Crude _FM.xlsx")
df

,Date,Month,Day,Year,Open,High,Low,Close
0,12/31/2019,12,31,2019,61.68,61.88,60.63,61.06
1,12/30/2019,12,30,2019,61.71,62.34,61.09,61.68
2,12/27/2019,12,27,2019,61.73,61.97,61.24,61.72
3,12/26/2019,12,26,2019,61.20,61.83,61.06,61.68
4,12/24/2019,12,24,2019,60.63,61.16,60.47,61.11
...,...,...,...,...,...,...,...,...
821,1/8/2018,1,8,2018,61.61,61.97,61.34,61.73
822,1/5/2018,1,5,2018,61.90,62.04,61.09,61.44
823,1/4/2018,1,4,2018,61.96,62.21,61.59,62.01
824,1/3/2018,1,3,2018,60.39,61.97,60.28,61.63


The float price of an Asian option is the average of the Future price of the front month contract over the period, i.e. the float price of an Apr21 Asian call option on WTI future is the average of the front month WTI future prices in Apr21



We have calculated the float price of the Asian option on WTI futures, since Apr21 we do not have the full month data, we would just drop the Apr21 data here.


In [21]:
average_price = df.groupby(["Year", "Month"])["Close"].mean()
average_price = average_price[0:-1:]
average_price

Year  Month
2018  1        63.659048
      2        62.183684
      3        62.771905
      4        66.325238
      5        69.983182
      6        67.322857
      7        70.581905
      8        67.845217
      9        70.084737
      10       70.757826
      11       56.693333
      12       48.777368
2019  1        51.550476
      2        54.980526
      3        58.168095
      4        63.870952
      5        60.768261
      6        54.706500
      7        57.537391
      8        54.844091
      9        56.878095
      10       54.005652
      11       57.070000
      12       59.784762
2020  1        57.528571
      2        50.664737
      3        30.445455
      4        20.282857
      5        28.527500
      6        38.313636
      7        40.765909
      8        42.388095
      9        39.625714
      10       39.554545
      11       41.346500
      12       47.068182
2021  1        52.149474
      2        59.061053
      3        62.357391
Name: Close, 

We calculate the first three moments and skewness of the observed float price data

In [22]:
m1_data = average_price.mean()
m2_data = (average_price**2).mean()
m3_data = (average_price**3).mean()
skew_data = skew(average_price)
print('%13s %14s' % ('statistic', 'Average Price of WTI Futures'))
print(45 * ".")
print('%20s %14.3f' % ('First Moment', m1_data))
print('%20s %14.3f' % ('Second Moment', m2_data))
print('%20s %14.3f' % ('Third Moment', m3_data))
print('%20s %14.3f' % ('Skew', skew_data))

    statistic Average Price of WTI Futures
.............................................
        First Moment         54.134
       Second Moment       3077.607
        Third Moment     181053.688
                Skew         -0.828


We first would like to match the observed two moments of the underlying average price/float price with log-normal distribution



In [23]:
def funtionsolve(z):
    mu=z[0]
    sigma=z[1]
    F=np.zeros(2)
    F[0] = math.exp(mu+1/2*sigma**2) - m1_data
    F[1] = math.exp(2*mu+2*sigma**2) - m2_data
    return F
zGuess = np.array([3,1])
z=fsolve(funtionsolve,zGuess)
print(z)

[3.96697559 0.22131019]


In [24]:
print('%35s' % ('lognormal parameters'))
print(50 * ".")
print('%20s %14.3f' % ('mean', z[0]))
print('%20s %14.3f' % ('sigma', z[1]))

               lognormal parameters
..................................................
                mean          3.967
               sigma          0.221


Now we could apply the Black76 formula as we approximate the underlying asian float price as lognormally distributed

In [31]:
def black_76(r,float,strike,sigma_float,T):
    d1 = (math.log(float/strike)+0.5*T*sigma_float**2)/(sigma_float*math.sqrt(T))
    d2 = d1-sigma_float*math.sqrt(T)
    c = math.exp(-r*T)*(float*norm.cdf(d1)-strike*norm.cdf(d2))
    return c

In [37]:
asian_float = 62
strike = 56
T=1/12
r=0.0166 ##use US 10-year treasury yield
a = black_76(r,asian_float,strike,0.221,T)
a

6.0798709609309505

Since the skew is negative and average price is always positive, we now match the moments of the underlying with shifted negative lognormal distribution 

In [38]:
def funtionsolve(z):
    tau = z[0]
    mu=z[1]
    sigma=z[2]
    F=np.zeros(3)
   
    F[0] = tau+ np.exp(mu+1/2*sigma**2) - 54.134
    F[1] = tau**2+ 2*tau*np.exp(mu+1/2*sigma**2) + np.exp(2*mu+2*sigma**2) - 3077.607
    F[2] = tau**3+ 3*tau**2*np.exp(mu+1/2*sigma**2) + 3*tau*np.exp(2*mu+2*sigma**2)+np.exp(3*mu+9/2*sigma**2) - 181053.688
    return F
zGuess = np.array([0,3,1])
z=fsolve(funtionsolve,zGuess)
print(z) 

[14.23716279  3.64329094  0.28543175]


In [39]:
tau = 14.23716279
mu = 3.64329094
sigma = 0.28543175


print('%20s %20s' % ('Estimated Value', 'True Value'))
print(45 * ".")
print('%16.3f %22.3f' % (tau+ np.exp(mu+1/2*sigma**2), 54.134))
print('%16.3f %22.3f' % (tau**2+ 2*tau*np.exp(mu+1/2*sigma**2) + np.exp(2*mu+2*sigma**2), 3077.607))
print('%16.3f %22.3f' % (tau**3+ 3*tau**2*np.exp(mu+1/2*sigma**2) + 3*tau*np.exp(2*mu+2*sigma**2)+np.exp(3*mu+9/2*sigma**2), 181053.688))

     Estimated Value           True Value
.............................................
          54.044                 54.134
        3055.201               3077.607
      181053.421             181053.688


In [40]:
tau = 14.23716279
mu = 3.64329094  
sigma = 0.28543175

asian_float = 62 - tau
strike = 56 - tau 
T=1/12
r=0.0166 ##use US 10-year treasury yield
b= black_76(r,asian_float,strike,sigma,T)
b

6.0713598870617975

In [41]:
print('%15s %30s' % ('Lognormal', 'Shifted Negative Lognormal'))
print(45 * ".")
print('%13.3f %22.3f' % (a, b))

      Lognormal     Shifted Negative Lognormal
.............................................
        6.080                  6.071
